In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.preprocessing.interpolation import Interpolation
import ptrail.utilities.constants as const

# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
gulls = pd.read_csv('./data/gulls.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='location-lat',
                        longitude='location-long',
                        datetime='timestamp',
                        traj_id='tag-local-identifier',
                        rest_of_columns=[])
gulls.head()

event-id  visible       lon       lat  \
traj_id DateTime                                                       
91732   2009-05-27 14:00:00  1082620685     True  24.58617  61.24783   
        2009-05-27 20:00:00  1082620686     True  24.58217  61.23267   
        2009-05-28 05:00:00  1082620687     True  24.53133  61.18833   
        2009-05-28 08:00:00  1082620688     True  24.58200  61.23283   
        2009-05-28 14:00:00  1082620689     True  24.58250  61.23267   

                            sensor-type individual-taxon-canonical-name  \
traj_id DateTime                                                          
91732   2009-05-27 14:00:00         gps                    Larus fuscus   
        2009-05-27 20:00:00         gps                    Larus fuscus   
        2009-05-28 05:00:00         gps                    Larus fuscus   
        2009-05-28 08:00:00         gps                    Larus fuscus   
        2009-05-28 14:00:00         gps                    Larus fuscus   

                            individual-local-identifier  \
traj_id DateTime                                          
91732   2009-05-27 14:00:00                      91732A   
        2009-05-27 20:00:00                      91732A   
        2009-05-28 05:00:00                      91732A   
        2009-05-28 08:00:00                      91732A   
        2009-05-28 14:00:00                      91732A   

                                                                    study-name  
traj_id DateTime                                                                
91732   2009-05-27 14:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-27 20:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 05:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 08:00:00  Navigation experiments in lesser black-backed ...  
        2009-05-28 14:00:00  Navigation experiments in lesser black-backed ...

In [2]:
t1 = gulls.reset_index().loc[gulls.reset_index()[const.TRAJECTORY_ID] == '91732'][[const.TRAJECTORY_ID, const.DateTime, const.LAT, const.LONG]]
t1 = PTRAILDataFrame(data_set=t1,
                     datetime='DateTime',
                     traj_id='traj_id',
                     latitude='lat',
                     longitude='lon')
t1.head()

lat       lon
traj_id DateTime                               
91732   2009-05-27 14:00:00  61.24783  24.58617
        2009-05-27 20:00:00  61.23267  24.58217
        2009-05-28 05:00:00  61.18833  24.53133
        2009-05-28 08:00:00  61.23283  24.58200
        2009-05-28 14:00:00  61.23267  24.58250

In [3]:
# ptdf = KinematicFeatures.generate_kinematic_features(t1)
# new_df = ptdf.reset_index()[['traj_id', 'Distance', 'Distance_from_start', 'Speed',
#                'Acceleration', 'Jerk', 'Bearing', 'Bearing_Rate',
#                'Rate_of_bearing_rate']]
# stats = new_df.reset_index(drop=True).describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).transpose()
# stats['traj_id'] = new_df['traj_id'].iloc[0]
# stats = stats[['traj_id','mean', 'std', 'min', '10%',
#                '25%', '50%', '75%', '90%','max']]
# stats.reset_index().rename(columns={'index': 'Columns'}).set_index(['traj_id', 'Columns'])

In [4]:
stats = KinematicFeatures.generate_kinematic_stats(gulls)

In [7]:
stats.head(16)

mean           std           min  \
traj_id Columns                                                          
91916   Distance              1.592839e+04  1.136323e+05  0.000000e+00   
        Distance_from_start   2.736727e+06  1.648074e+06  2.498220e+02   
        Speed                 8.464983e-01  1.128293e+01  0.000000e+00   
        Acceleration          1.669456e-05  1.440443e-03 -3.176928e-02   
        Jerk                  1.643145e-09  2.277868e-07 -1.102664e-05   
        Bearing               1.875603e+02  1.076184e+02  0.000000e+00   
        Bearing_Rate         -2.545166e-04  8.810098e-03 -3.791598e-02   
        Rate_of_bearing_rate -2.545166e-04  8.810098e-03 -3.791598e-02   
91752   Distance              6.084898e+03  1.945547e+04  0.000000e+00   
        Distance_from_start   5.723643e+06  1.296516e+06  3.510347e+02   
        Speed                 3.178652e-01  8.412081e-01  0.000000e+00   
        Acceleration          3.831443e-06  3.812527e-05 -8.603067e-04   
        Jerk                  3.630524e-10  3.814786e-09 -8.933501e-08   
        Bearing               1.680275e+02  1.044518e+02  0.000000e+00   
        Bearing_Rate         -7.857827e-05  9.403411e-03 -3.305716e-02   
        Rate_of_bearing_rate -7.857827e-05  9.403411e-03 -3.305716e-02   

                                       10%           25%           50%  \
traj_id Columns                                                          
91916   Distance              1.996660e+01  5.017081e+02  3.015994e+03   
        Distance_from_start   9.137111e+05  1.253106e+06  2.168522e+06   
        Speed                 1.158369e-03  2.522285e-02  1.416421e-01   
        Acceleration         -4.256941e-05 -1.017532e-05 -2.819263e-08   
        Jerk                 -3.744878e-09 -7.889089e-10  4.641791e-11   
        Bearing               3.017762e+01  1.056518e+02  1.800000e+02   
        Bearing_Rate         -1.109072e-02 -6.410844e-03  0.000000e+00   
        Rate_of_bearing_rate -1.109072e-02 -6.410844e-03  0.000000e+00   
91752   Distance              4.092670e+01  3.146816e+02  2.326454e+03   
        Distance_from_start   6.084663e+06  6.090395e+06  6.091126e+06   
        Speed                 1.907036e-03  1.487929e-02  1.098361e-01   
        Acceleration         -2.286116e-05 -5.999053e-06 -4.583517e-08   
        Jerk                 -2.217735e-09 -4.317609e-10  6.455114e-11   
        Bearing               2.447125e+01  7.534143e+01  1.734714e+02   
        Bearing_Rate         -1.136807e-02 -6.263162e-03  1.540565e-05   
        Rate_of_bearing_rate -1.136807e-02 -6.263162e-03  1.540565e-05   

                                       75%           90%           max  
traj_id Columns                                                         
91916   Distance              1.169823e+04  2.989838e+04  6.587322e+06  
        Distance_from_start   4.514151e+06  4.514742e+06  8.446012e+06  
        Speed                 6.039781e-01  1.389204e+00  9.149059e+02  
        Acceleration          9.528719e-06  5.814845e-05  1.270143e-01  
        Jerk                  1.494163e-09  5.204901e-09  1.764004e-05  
        Bearing               2.859466e+02  3.341752e+02  3.597854e+02  
        Bearing_Rate          5.573409e-03  9.708628e-03  3.260693e-02  
        Rate_of_bearing_rate  5.573409e-03  9.708628e-03  3.260693e-02  
91752   Distance              7.319771e+03  1.489576e+04  5.601945e+05  
        Distance_from_start   6.092304e+06  6.102565e+06  6.113107e+06  
        Speed                 3.718564e-01  8.023213e-01  1.728996e+01  
        Acceleration          6.429819e-06  3.387030e-05  5.968416e-04  
        Jerk                  9.541063e-10  3.103903e-09  4.704470e-08  
        Bearing               2.596122e+02  3.103268e+02  3.599153e+02  
        Bearing_Rate          6.740900e-03  1.076744e-02  3.222234e-02  
        Rate_of_bearing_rate  6.740900e-03  1.076744e-02  3.222234e-02